<h3>3.1 Acciones iniciales</h3>

In [1]:
!pip install boto3       # Librería acceso AWS
!pip install Faker       # Generar datos fake 

In [3]:
import boto3
import botocore
import os
import time

ts_actual = int(time.time())

In [35]:
ts_actual

1683710007

In [5]:
dynamo_client = boto3.client('dynamodb', region_name='eu-west-3')

In [6]:
dynamo_client

<h3>3.2 Listar y describir tablas</h3>

In [9]:
# Listar las tablas que tengamos en nuestro entorno
response = dynamo_client.list_tables()
for table in response['TableNames']:    
    print(f'Detalles de la tabla {table}\n')
    print(f'{dynamo_client.describe_table(TableName=table)}\n')          
    # Mostramos la información de cada tabla aunque no tenga datos

Detalles de la tabla Ejercicio

{'Table': {'AttributeDefinitions': [{'AttributeName': 'Especialidad', 'AttributeType': 'S'}, {'AttributeName': 'Registro', 'AttributeType': 'N'}, {'AttributeName': 'Tipo', 'AttributeType': 'S'}], 'TableName': 'Ejercicio', 'KeySchema': [{'AttributeName': 'Tipo', 'KeyType': 'HASH'}, {'AttributeName': 'Registro', 'KeyType': 'RANGE'}], 'TableStatus': 'ACTIVE', 'CreationDateTime': datetime.datetime(2023, 5, 10, 11, 11, 52, 838000, tzinfo=tzlocal()), 'ProvisionedThroughput': {'NumberOfDecreasesToday': 0, 'ReadCapacityUnits': 0, 'WriteCapacityUnits': 0}, 'TableSizeBytes': 0, 'ItemCount': 0, 'TableArn': 'arn:aws:dynamodb:eu-west-3:336247090034:table/Ejercicio', 'TableId': 'bbc52b74-fc1b-48d6-ada8-f7f9a2d2953e', 'BillingModeSummary': {'BillingMode': 'PAY_PER_REQUEST', 'LastUpdateToPayPerRequestDateTime': datetime.datetime(2023, 5, 10, 11, 11, 52, 838000, tzinfo=tzlocal())}, 'LocalSecondaryIndexes': [{'IndexName': 'Especialidad-index', 'KeySchema': [{'AttributeNam

In [10]:
response

{'TableNames': ['Ejercicio'],
 'ResponseMetadata': {'RequestId': 'H5RLH1NKTTBLBEU42KIOKQ5RP7VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 10 May 2023 09:19:03 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '28',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'H5RLH1NKTTBLBEU42KIOKQ5RP7VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '512678937'},
  'RetryAttempts': 0}}

<h3>3.3 Añadiendo entradas aisladas</h3>

Aqui usamos otra tabla para entender mejor como añadir entrada

In [15]:
# dynamo_client = boto3.client('dynamodb', region_name='eu-west-3')

TABLE_NAME = 'Modulo11'
entrada = {
    "Tipo": "Profesor",
    "Registro": ts_actual,
    "Especialidad": "AWS"
}

try:
    put_response = dynamo_client.put_item(TableName=TABLE_NAME, Item=entrada)
    print(f'0.-Entrada {entrada} añadida correctamente: {put_response} \n')
except Exception as e:
    print(f'0.-Problema al insertar la entrada {entrada}: {e} \n')
# Es normal que no funcione

0.-Problema al insertar la entrada {'Tipo': 'Profesor', 'Registro': 1683710007, 'Especialidad': 'AWS'}: Parameter validation failed:
Invalid type for parameter Item.Tipo, value: Profesor, type: <class 'str'>, valid types: <class 'dict'>
Invalid type for parameter Item.Registro, value: 1683710007, type: <class 'int'>, valid types: <class 'dict'>
Invalid type for parameter Item.Especialidad, value: AWS, type: <class 'str'>, valid types: <class 'dict'> 



-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [28]:
# Vemos que se nos solicita un formato específico para los datos --> Comprobar en la consola usando DynamoDB JSON
# S=straing y N=number
entrada = {
    "Tipo": {"S": "Profesor"},
    "Registro": {"N": str(ts_actual)},
    "Especialidad": {"S": "AWS"}
}


try:
    put_response = dynamo_client.put_item(TableName=TABLE_NAME, Item=entrada)
    print(f'1.- Entrada {entrada} añadida correctamente: {put_response} \n')
    print(f'1.- Numero total de entradas: {dynamo_client.describe_table(TableName=TABLE_NAME)["Table"]["ItemCount"]}\n')     
except Exception as e:
    print(f'1.- Problema al insertar la entrada {entrada}: {e} \n')   

1.- Entrada {'Tipo': {'S': 'Profesor'}, 'Registro': {'N': '1683710007'}, 'Especialidad': {'S': 'AWS'}} añadida correctamente: {'ResponseMetadata': {'RequestId': 'L6UGGJI9IVVVBLMMBOGFDRTPS3VV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 10 May 2023 10:16:51 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': 'L6UGGJI9IVVVBLMMBOGFDRTPS3VV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}, 'RetryAttempts': 0}} 

1.- Numero total de entradas: 0



-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [30]:
# Si repetimos la operación, aunque no falle (codigo 200) , lo que estamos haciendo es "sobreescribir la entrada previa" y 
# no cambia nada (por eso de que sigeteniendo el mismo valor) 

for i in range(1,6):
    try:
        put_response = dynamo_client.put_item(TableName=TABLE_NAME, Item=entrada)
        print(f'2.{i}.- Entrada {entrada} añadida correctamente: {put_response} \n')
        print(f'2.{i}.- Numero total de entradas: {dynamo_client.describe_table(TableName=TABLE_NAME)["Table"]["ItemCount"]}\n')     
    except Exception as e:
        print(f'2-{i}.- Problema al insertar la entrada {entrada}: {e} \n')       

2.1.- Entrada {'Tipo': {'S': 'Profesor'}, 'Registro': {'N': '1683710007'}, 'Especialidad': {'S': 'AWS'}} añadida correctamente: {'ResponseMetadata': {'RequestId': 'B02S1H1T92R6O7OK40PJJ5U30NVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 10 May 2023 10:18:08 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': 'B02S1H1T92R6O7OK40PJJ5U30NVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}, 'RetryAttempts': 0}} 

2.1.- Numero total de entradas: 0

2.2.- Entrada {'Tipo': {'S': 'Profesor'}, 'Registro': {'N': '1683710007'}, 'Especialidad': {'S': 'AWS'}} añadida correctamente: {'ResponseMetadata': {'RequestId': '3DN981RPBVJNKB6N63GLK61R73VV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 10 May 2023 10:18:08 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-a

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [29]:
# Podemos usar un parámetro para obtener el resultado, de forma que podemos comprobar si se ha sobreescrito o no

try:
    put_response = dynamo_client.put_item(TableName=TABLE_NAME, Item=entrada, ReturnValues="ALL_OLD")
    print(f'3.- Entrada {entrada} añadida correctamente: {put_response} \n')
    print(f'3.- Numero total de entradas: {dynamo_client.describe_table(TableName=TABLE_NAME)["Table"]["ItemCount"]}\n')     
except Exception as e:
    print(f'3.-Problema al insertar la entrada {entrada}: {e} \n')

3.- Entrada {'Tipo': {'S': 'Profesor'}, 'Registro': {'N': '1683710007'}, 'Especialidad': {'S': 'AWS'}} añadida correctamente: {'Attributes': {'Tipo': {'S': 'Profesor'}, 'Especialidad': {'S': 'AWS'}, 'Registro': {'N': '1683710007'}}, 'ResponseMetadata': {'RequestId': 'OMPAF83M2G7RCM8DDDBG92U14FVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 10 May 2023 10:16:56 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '97', 'connection': 'keep-alive', 'x-amzn-requestid': 'OMPAF83M2G7RCM8DDDBG92U14FVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '1076581699'}, 'RetryAttempts': 0}} 

3.- Numero total de entradas: 0



------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [21]:
# Podemos también usar una condición para que solo se esciba si no existe previamente

CONDICION = 'attribute_not_exists(Tipo) AND attribute_not_exists(Registro)'

try:
    put_response = dynamo_client.put_item(TableName=TABLE_NAME, Item=entrada, ConditionExpression=CONDICION)
    print(f'4.-Entrada {entrada} añadida correctamente: {put_response} \n')
    print(f'4.- Numero total de entradas: {dynamo_client.describe_table(TableName=TABLE_NAME)["Table"]["ItemCount"]}\n')     
except Exception as e:
    print(f'4.- Problema al insertar la entrada {entrada}: {e} \n') 

4.- Problema al insertar la entrada {'Tipo': {'S': 'Profesor'}, 'Registro': {'N': '1683710007'}, 'Especialidad': {'S': 'AWS'}}: An error occurred (ConditionalCheckFailedException) when calling the PutItem operation: The conditional request failed 



-------------------------------------------------------------------------------------------------------------------------------------------------------------------------

De forma simular a put_item podemos utilizar update_item.
La diferencia entre ambos métodos es la siguiente:
* put_item: si existe una entrada anterior con las mismas claves la reemplaza
    * Se eliminan los campos no proporcionados
* update_item: si existe una entrada anterior con las mismas claves la actualiza
    * Se mantienen los campos no proporcionados, los proporcionados se actualizan

<h3>3.4 Añadiendo entradas en batch</h3>

<h5>El método batch_write_item  permite realizar una carga "masiva" de datos, reduciendo la latencia al no tener que
establecer una pequeña conexión por cada escritura, además de realizar las escrituras en paralelo.
Que podemos crear varios elementos.
Con esto no podemos actualizar entradas anteriores, y tenemos limitaciones al no poder enviar más de 16Mb de datos por acción o más de 25 operaciones. </h5>

In [25]:
response = dynamo_client.batch_write_item(RequestItems={
    TABLE_NAME: [
        {
            "PutRequest": {
                "Item": {
                    "Tipo": {"S": "Alumno"},
                    "Registro": {"N": str(ts_actual+1)},
                    "Especialidad": {"S": "AWS"}
                }
            }
        },
        {
            "PutRequest": {
                "Item": {
                    "Tipo": {"S": "Alumno"},
                    "Registro": {"N": str(ts_actual+2)},
                    "Especialidad": {"S": "AWS"}
                }
            }
        }
        ,
        {
            "PutRequest": {
                "Item": {
                    "Tipo": {"S": "Alumno"},
                    "Registro": {"N": str(ts_actual+3)},
                    "Especialidad": {"S": "AWS"}
                }
            }
        }
    ]
},ReturnConsumedCapacity="TOTAL",ReturnItemCollectionMetrics="SIZE")

print(response)
print(f'Numero total de entradas: {dynamo_client.describe_table(TableName=TABLE_NAME)["Table"]["ItemCount"]}\n')     


{'UnprocessedItems': {}, 'ItemCollectionMetrics': {'Modulo11': [{'ItemCollectionKey': {'Tipo': {'S': 'Alumno'}}, 'SizeEstimateRangeGB': [0.0, 1.0]}]}, 'ConsumedCapacity': [{'TableName': 'Modulo11', 'CapacityUnits': 6.0}], 'ResponseMetadata': {'RequestId': 'VN0R07I1TQ5C2C7V19N3EB3POBVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 10 May 2023 10:08:33 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '206', 'connection': 'keep-alive', 'x-amzn-requestid': 'VN0R07I1TQ5C2C7V19N3EB3POBVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '3596598752'}, 'RetryAttempts': 0}}
Numero total de entradas: 0



-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [37]:
actions_for_table = []

N_UPDATES = 25 # No puede ser mayor que 25

for i in range(0,N_UPDATES):
    put_request = {
        "PutRequest": {
            "Item": {
                "Tipo": {"S": "Alumno"},
                "Registro": {"N": str(ts_actual+i)},
                "Especialidad": {"S": "AWS"}
            }
        }
    }
    actions_for_table.append(put_request)   

try:
    response = dynamo_client.batch_write_item(RequestItems={
        TABLE_NAME: actions_for_table
    },ReturnConsumedCapacity="TOTAL",ReturnItemCollectionMetrics="SIZE")
except Exception as e:
    print(f'\n\nProblema durante la escritura batch: {e}\n\n')

print(response)
print(f'Numero total de entradas: {dynamo_client.describe_table(TableName=TABLE_NAME)["Table"]["ItemCount"]}\n')   

{'UnprocessedItems': {}, 'ItemCollectionMetrics': {'Modulo11': [{'ItemCollectionKey': {'Tipo': {'S': 'Alumno'}}, 'SizeEstimateRangeGB': [0.0, 1.0]}]}, 'ConsumedCapacity': [{'TableName': 'Modulo11', 'CapacityUnits': 45.0}], 'ResponseMetadata': {'RequestId': 'DQDHT3A8EAGTQL86HL0Q11SDGBVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 10 May 2023 10:28:23 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '207', 'connection': 'keep-alive', 'x-amzn-requestid': 'DQDHT3A8EAGTQL86HL0Q11SDGBVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '1001552310'}, 'RetryAttempts': 0}}
Numero total de entradas: 0



--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Este codigo probocara que se ejecute pero cuanto mas escriba mar errores tendra y menos creara. Lo llevara a los limite de la escritura, probocando errores

In [ ]:
N_UPDATES_PER_EXECUTION = 25
N_ITERATIONS = 20

for j in range(0,N_ITERATIONS):
    actions_for_table = []    
    ts_actual = int(time.time())
    for i in range(0,N_UPDATES_PER_EXECUTION):
        put_request = {
            "PutRequest": {
                "Item": {
                    "Tipo": {"S": "Alumno"},
                    "Registro": {"N": str(ts_actual+i)},
                    "Especialidad": {"S": "AWS"}
                }
            }
        }
        actions_for_table.append(put_request)   
    try:
        response = dynamo_client.batch_write_item(RequestItems={
            TABLE_NAME: actions_for_table
        },ReturnConsumedCapacity="TOTAL",ReturnItemCollectionMetrics="SIZE")
    except Exception as e:
        print(f'\n\nProblema durante la escritura batch: {e}\n\n')
        

    print(response)

<h3>3.5 Realizando consultas</h3>

A pesar de tener el método scan, se desaconseja su uso dado que DynamoDB tendrá que explorar todo el conjunto de datos antes de devolver el resultado deseado, lo cual afecta a las capacidades de lectura.

Usaremos query() 

Debemos diferenciar entre condiciones sobre la clave o filtrados:
* Condiciones sobre la clave: determinan los datos que se van a extraer de la base de datos
* Filtrados: sobre los datos extraidos de la base de datos, determina los que se proporcionan con la respuesta

In [41]:
# Como mínimo, además de la tabla debemos proporcionar una condición a través de KeyConditions o KeyConditionExpression

CONDICION = 'Tipo = :tipo'
ATRIBUTOS_CONDICION = {':tipo': {'S': 'Profesor'}}

try:
    response = dynamo_client.query(TableName=TABLE_NAME, KeyConditionExpression=CONDICION, ExpressionAttributeValues=ATRIBUTOS_CONDICION)
    print(f'{response}\n\n')
    print(f'Respuesta con {response["Count"]} registros\n')    
    for item in response.get('Items', []):
        print(f'{item}\n')
except Exception as e:
    print(f'Problema durante la lectura: {e}')


{'Items': [{'Tipo': {'S': 'Profesor'}, 'Especialidad': {'S': 'AWS'}, 'Registro': {'N': '1683710007'}}], 'Count': 1, 'ScannedCount': 1, 'ResponseMetadata': {'RequestId': '5GN3L08C1PNN61TJ2EONJ8SRLVVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 10 May 2023 10:51:08 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '121', 'connection': 'keep-alive', 'x-amzn-requestid': '5GN3L08C1PNN61TJ2EONJ8SRLVVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '180727035'}, 'RetryAttempts': 0}}


Respuesta con 1 registros

{'Tipo': {'S': 'Profesor'}, 'Especialidad': {'S': 'AWS'}, 'Registro': {'N': '1683710007'}}



-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [42]:
# Además podemos aplicar filtrado (se aplica tras obtener los datos de la tabla)

CONDICION = 'Tipo = :tipo'
FILTRADO = 'Ciudad = :ciudad' 

ATRIBUTOS_CONDICION = {
    ':tipo': {'S': 'Profesor'},
    ':ciudad': {'S': 'Sevilla'},
}

try:
    response = dynamo_client.query(
        TableName=TABLE_NAME, 
        KeyConditionExpression=CONDICION, 
        FilterExpression=FILTRADO,
        ExpressionAttributeValues=ATRIBUTOS_CONDICION)
    print(f'{response}\n\n')
    print(f'Respuesta con {response["Count"]} registros\n')    
    for item in response.get('Items', []):
        print(f'{item}\n')
except Exception as e:
    print(f'Problema durante la lectura: {e}')


{'Items': [], 'Count': 0, 'ScannedCount': 1, 'ResponseMetadata': {'RequestId': 'H5P3H4170ESJJT37VP0DGOR4GJVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 10 May 2023 10:52:01 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '39', 'connection': 'keep-alive', 'x-amzn-requestid': 'H5P3H4170ESJJT37VP0DGOR4GJVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '3530536553'}, 'RetryAttempts': 0}}


Respuesta con 0 registros



------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
Se puede proporcionar un valor específico de Sort Key para obtener un único resultado

In [ ]:
# En la condicion, podemos añadir el valor la Sort Key para reducir aún más el resultado

CONDICION = 'Tipo = :tipo and Registro = :registro'
ATRIBUTOS_CONDICION = {':tipo': {'S': 'Profesor'}, ':registro': {'N': '0'}}

try:
    response = dynamo_client.query(TableName=TABLE_NAME, KeyConditionExpression=CONDICION, ExpressionAttributeValues=ATRIBUTOS_CONDICION)
    print(f'{response}\n\n')
    print(f'Respuesta con {response["Count"]} registros\n')    
    for item in response.get('Items', []):
        print(f'{item}\n')
except Exception as e:
    print(f'Problema durante la lectura: {e}')

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
O trabajar con condiciones sobre la Sort Key

In [ ]:
ts_hace_una_hora = int(time.time()) - 60*60
CONDICION = 'Tipo = :tipo and Registro < :ts_hace_una_hora'
ATRIBUTOS_CONDICION = {':tipo': {'S': 'Alumno'}, ':ts_hace_una_hora': {'N': str(ts_hace_una_hora)}}

try:
    response = dynamo_client.query(TableName=TABLE_NAME, KeyConditionExpression=CONDICION, ExpressionAttributeValues=ATRIBUTOS_CONDICION)
    print(f'{response}\n\n')
    print(f'Respuesta con {response["Count"]} registros\n')    
    for item in response.get('Items', []):
        print(f'{item}\n')
except Exception as e:
    print(f'Problema durante la lectura: {e}')